# Leaflet cluster map of talk locations

Run this from the _talks/ directory, which contains .md files of all your talks. This scrapes the location YAML field from each .md file, geolocates it with geopy/Nominatim, and uses the getorg library to output data, HTML, and Javascript for a standalone cluster map.

In [1]:
!pip install getorg --upgrade
import glob
import getorg
from geopy import Nominatim

Iywidgets and ipyleaflet support disabled. You must be in a Jupyter notebook to use this feature.
Error raised:
No module named 'ipyleaflet'
Check that you have enabled ipyleaflet in Jupyter with:
    jupyter nbextension enable --py ipyleaflet


In [2]:
g = glob.glob("*.md")

In [3]:
geocoder = Nominatim(user_agent="myGeocoder")
location_dict = {}
location = ""
permalink = ""
title = ""

In [4]:

for file in g:
    with open(file, 'r') as f:
        lines = f.read()
        if lines.find('location: "') > 1:
            loc_start = lines.find('location: "') + 11
            lines_trim = lines[loc_start:]
            loc_end = lines_trim.find('"')
            location = lines_trim[:loc_end]
                            
           
        location_dict[location] = geocoder.geocode(location)
        print(location, "\n", location_dict[location])


 
 None


 
 None


In [5]:
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="../talkmap", hashed_usernames=False)

'Written map to ../talkmap/'